<span class='note'>*Make me look good.* Click on the cell below and press <kbd>Ctrl</kbd>-<kbd>Enter</kbd>.</span>

In [1]:
from IPython.core.display import HTML
HTML(open('css/custom.css', 'r').read())

<h5 class='prehead'>SA367 &middot; Mathematical Models for Decision Making &middot; Spring 2020 &middot; Uhan</h5>

<h5 class='lesson'>Lesson 15.</h5>

<h1 class='lesson_title'>Solving stochastic dynamic programs with Python</h1>

## Overview

* Let's solve the stochastic dynamic program we formulated for the investment problem in Lesson 14.
* In this class, we will use a package called `stochasticdp` to set up and solve stochastic dynamic programs.
    - _Warning._ This is a package that I wrote. There may be some bugs.
    - _Note._ This package is publicly available. Please feel free to use it in the future for other things. The source code is on [GitHub](https://github.com/nelsonuhan/stochasticdp).

## Installing `stochasticdp`

* To install `stochasticdp`, open a WinPython Command Prompt and type:

```
pip install stochasticdp
```

* To use `stochasticdp`, we must first import it. In `stochasticdp`, we only need the object `StochasticDP`, so we can perform our import like this:

In [2]:
from stochasticdp import StochasticDP

## Setting up a stochastic dynamic program

* Recall the investment problem from Lesson 18:

__Problem.__ Suppose you have \\$5,000 to invest. Over the next 3 years, you want to double your money.
At the beginning of each of the next 3 years, you have an opportunity to invest in one of two investments: A or B. Both investments have uncertain profits. For an investment of \\$5,000, the profits are as follows:


| Investment | Profit (\$) | Probability |
|:-----------|------------:|------------:|
| A          | -5,000      | 0.3         |
|            | 5,000       | 0.7         |
| B          | 0           | 0.9         |
|            | 5,000       | 0.1         |
                                     
You are allowed to make at most one investment each year, and can invest only \\$5,000 each time. Any additional money accumulated is left idle. Once you've accumulated \\$10,000, you stop investing.

Formulate a stochastic dynamic program to find an investment policy that maximizes the probability you will have \$10,000 after 3 years.

* Let's walk through setting up the stochastic DP we formulated in the last lesson.

### Initialization

* We had defined 4 stages.

* To make things easier, let's renumber the stages so they start at $t = 0$:

$$
\begin{aligned}
\text{stage } t = 0, 1, 2 & \quad\leftrightarrow\quad \text{beginning of year $t$}\\
t = 3 & \quad\leftrightarrow\quad \text{end of process}
\end{aligned}
$$
    
* In each stage, we defined 3 states:

$$
\text{state } n \in \{0, 5000, 10000\} \quad\leftrightarrow\quad \text{$n$ dollars in account}
$$

* At each stage and state, we defined 3 possible decisions:

$$
\text{decision } x_t \in \{ \text{A}, \text{B}, \text{no investment} \}
$$

* The set of _allowable_ decisions changed, depending on the stage and state. We'll address this later.

* For now, we can initialize a stochastic dynamic program with these stages, states, and decisions like this:

In [3]:
# Number of stages
number_of_stages = 4

# List of states
states = [0, 5000, 10000]

# List of decisions
decisions = ['A', 'B', 'no investment']

# Initialize stochastic dynamic program - we want to maximize, so minimize = False
dp = StochasticDP(number_of_stages, states, decisions, minimize=False)

* The code above initializes a stochastic dynamic program called `dp`.

* Next, we need to add every transition that occurs with positive probability.

### Transition probabilities and contributions

* First, let's tackle transitions from the state $n = 5000$:

<center><img src="img/5000.png" width="60%"></center>

* We can add a transition from state $n$ to state $m$ in stage $t$ under decision $x$ with probability $p(m \,|\, n, t, x)$ and contribution $c(m \,|\, n, t, x)$ as follows:

```python
dp.add_transition(stage=t, from_state=n, decision=x, to_state=n, probability=p, contribution=c)
```

* So, we can input the transition probabilities and contributions from state $n = 5000$ in stages $t = 0, 1, 2$ as follows.

    * Remember that the contributions for all transitions are 0 in this stochastic DP.

In [4]:
# Transition probabilities and contributions from state n = 5000
for t in range(number_of_stages - 1):
    # Investment A
    dp.add_transition(stage=t, from_state=5000, decision='A', to_state=10000, probability=0.7, contribution=0)
    dp.add_transition(stage=t, from_state=5000, decision='A', to_state=0, probability=0.3, contribution=0)

    # Investment B
    dp.add_transition(stage=t, from_state=5000, decision='B', to_state=10000, probability=0.1, contribution=0)
    dp.add_transition(stage=t, from_state=5000, decision='B', to_state=5000, probability=0.9, contribution=0)

    # No investment
    dp.add_transition(stage=t, from_state=5000, decision='no investment', to_state=5000, probability=1, contribution = 0)

* _Quick check._ What can the sum of the transition probabilities from any decision node equal?

<!-- _Write your notes here. Double-click to edit._ -->
The transition probabilities from any decision node must add up to either 0 or 1. They will add up to 1 if the decision is allowable at that stage/state; otherwise they will add up to 0.

* Next, let's tackle the transitions from state $n = 0$:

![n = 0](img/0.png)

* So, we can input the transition probabilities and contributions from state $n = 0$ in stages $t = 0, 1, 2$ like this:

In [5]:
# Transition probabilities and contributions from state n = 0
for t in range(number_of_stages - 1):
    # No investment
    dp.add_transition(stage=t, from_state=0, decision='no investment', to_state=0, probability=1, contribution = 0)

* We can tackle the transitions from state $n = 10000$ in an almost identical way:

![n = 10000](img/10000.png)

In [6]:
# Transition probabilities and contributions from state n = 10000
for t in range(number_of_stages - 1):
    # No investment
    dp.add_transition(stage=t, from_state=10000, decision='no investment', to_state=10000, probability=1, contribution = 0)

### Boundary conditions

* Finally, we need to define the boundary conditions.

* In particular, we need to specify the value-to-go function at the last stage (in our case, $t = 3$) for each state.

* We can add the boundary conditions for state $n$ like this:

```python
dp.add_boundary(state=0, value=0)
```

* So, let's add the boundary conditions for our problem:

In [7]:
# Boundary conditions
dp.add_boundary(state=10000, value=1)
dp.add_boundary(state=5000, value=0)
dp.add_boundary(state=0, value=0)

## Solving the stochastic dynamic program

* Once the stochastic DP is setup, we can solve it like this:

In [8]:
# Solve the stochastic dynamic program
value, policy = dp.solve()

* Note that the method `.solve()` outputs two objects: `value` and `policy`.

* `value[t, n]` is the value-to-go function $f_t(n)$ at stage $t$ and state $n$.

* `policy[t, n]` is the optimal decision $x_t^*$ that attains the value-to-go function $f_t(n)$ at stage $t$ and state $n$.

* First, let's see what the value-to-go function looks like:

In [9]:
# Examine the value-to-go function
print(value)

{(stage: 0, state: 0): 0
 (stage: 0, state: 5000): 0.757
 (stage: 0, state: 10000): 1
 (stage: 1, state: 0): 0
 (stage: 1, state: 5000): 0.73
 (stage: 1, state: 10000): 1
 (stage: 2, state: 0): 0
 (stage: 2, state: 5000): 0.7
 (stage: 2, state: 10000): 1
 (stage: 3, state: 0): 0
 (stage: 3, state: 5000): 0
 (stage: 3, state: 10000): 1}


* Next, let's look at the corresponding policy:

In [10]:
# Examine the policy
print(policy)

{(stage: 0, state: 0): {'no investment'}
 (stage: 0, state: 5000): {'B'}
 (stage: 0, state: 10000): {'no investment'}
 (stage: 1, state: 0): {'no investment'}
 (stage: 1, state: 5000): {'B'}
 (stage: 1, state: 10000): {'no investment'}
 (stage: 2, state: 0): {'no investment'}
 (stage: 2, state: 5000): {'A'}
 (stage: 2, state: 10000): {'no investment'}}


## On your own

* Solve the stochastic DP we formulated in Lesson 14 for this problem:

__Problem.__ The Hit-and-Miss Manufacturing Company has received an order to supply one item of a particular type. However, manufacturing this item is difficult, and the customer has specified such stringent quality requirements that the company may have to produce more than one item to obtain an item that is acceptable.
                
The company estimates that each item of this type will be acceptable with probability 1/2 and defective with probability 1/2. Each item costs \\$100 to produce, and excess items are worthless. In addition, a setup cost of \\$300 must be incurred whenever the production process is setup for this item. The company has time to make no more than 3 production runs, and at most 5 items can be produced in each run. If an acceptable item has not been obtained by the end of the third production run, the manufacturer is in breach of contract and must pay a penalty of \\$1600.
                
The objective is to determine how many items to produce in each production run in order to minimize the total expected cost.

In [11]:
# Number of stages
number_of_stages = 4

# List of states
states = [0, 1]

# List of decisions
decisions = [0, 1, 2, 3, 4, 5]

# Initialize stochastic dynamic program
dp = StochasticDP(number_of_stages, states, decisions, minimize=True)

# Transition probabilities and contributions from state n = 0
for t in range(number_of_stages - 1):
    for x in decisions:
        if x > 0:
            K = 300
        else:
            K = 0
            
        dp.add_transition(stage=t, from_state=0, decision=x, to_state=0, probability=1, contribution=K + 100*x)
        dp.add_transition(stage=t, from_state=0, decision=x, to_state=1, probability=0, contribution=K + 100*x)        

# Transition probabilities and contributions from state n = 1
for t in range(number_of_stages - 1):
    for x in decisions:
        if x > 0:
            K = 300
        else:
            K = 0
            
        dp.add_transition(stage=t, from_state=1, decision=x, to_state=0, probability=1 - (1/2)**x, contribution=K + 100*x)
        dp.add_transition(stage=t, from_state=1, decision=x, to_state=1, probability=(1/2)**x, contribution=K + 100*x)
        
# Boundary conditions
dp.add_boundary(state=0, value=0)
dp.add_boundary(state=1, value=1600)

# Solve the stochastic dynamic program
value, policy = dp.solve()

# Examine value-to-go
print("The value-to-go function is:")
print(value)

# Examine policy
print("\nThe corresponding policy is:")
print(policy)

The value-to-go function is:
{(stage: 0, state: 0): 0
 (stage: 0, state: 1): 675.0
 (stage: 1, state: 0): 0
 (stage: 1, state: 1): 700.0
 (stage: 2, state: 0): 0
 (stage: 2, state: 1): 800.0
 (stage: 3, state: 0): 0
 (stage: 3, state: 1): 1600}

The corresponding policy is:
{(stage: 0, state: 0): {0}
 (stage: 0, state: 1): {2}
 (stage: 1, state: 0): {0}
 (stage: 1, state: 1): {2, 3}
 (stage: 2, state: 0): {0}
 (stage: 2, state: 1): {3, 4}}


---

## Problems

### Problem 1 (Baytheon, revisited)

Baytheon has received an order to supply 2 guided missiles. In order to meet stringent quality requirements, the company may have to manufacture more than one missile to obtain an missile that is acceptable. The company has time to make no more than 3 production runs, and at most 2 missiles can be produced in each run. The probability distribution of acceptable missiles in a given run depends on how many missiles are produced:

| Number of missiles produced | Prob. of 0 acceptable missiles | Prob. of 1 acceptable missile | Prob. of 2 acceptable missiles |
|-----------------------------|--------------------------------|-------------------------------|--------------------------------| 
| 0                           | 1                              | 0                             | 0                              |
| 1                           | 1/3                            | 2/3                           | 0                              |
| 2                           | 1/4                            | 1/2                           | 1/4                            |
    
Each missile costs \\$100,000 to produce, and excess missiles are worthless. In addition, a setup cost of \\$50,000 must be incurred whenever the production process is setup for this item. If 2 acceptable missiles have not been obtained by the end of the third production run, Baytheon is in breach of contract and must pay a penalty of \\$1,000,000. The objective is to determine how many missiles to produce in each production run in order to minimize the total expected cost.

Once upon a time, for homework, you formulated this problem as a stochastic dynamic program. (Note that the penalty value has changed.)

1. Solve your dynamic program using `stochasticdp`.
2. Interpret the output of your stochastic dynamic program.

In [18]:
# Solution
# Number of stages
number_of_stages = 4

# List of states
states = [0, 1, 2]

# List of decisions
decisions = [0, 1, 2]

# Initialize stochastic dynamic program
dp = StochasticDP(number_of_stages, states, decisions, minimize=True)

# Transition probabilities and contributions from state n = 2
for t in range(number_of_stages - 1):
    # Produce 2
    dp.add_transition(stage=t, from_state=2, decision=2, to_state=2, probability=1/4, contribution=50 + 100*2)
    dp.add_transition(stage=t, from_state=2, decision=2, to_state=1, probability=1/2, contribution=50 + 100*2)
    dp.add_transition(stage=t, from_state=2, decision=2, to_state=0, probability=1/4, contribution=50 + 100*2)
    
    # Produce 1
    dp.add_transition(stage=t, from_state=2, decision=1, to_state=2, probability=1/3, contribution=50 + 100*1)
    dp.add_transition(stage=t, from_state=2, decision=1, to_state=1, probability=2/3, contribution=50 + 100*1)
    
    # Produce 0
    dp.add_transition(stage=t, from_state=2, decision=0, to_state=2, probability=1, contribution=0)
    
# Transition probabilities and contributions from state n = 1
for t in range(number_of_stages - 1):
    # Produce 2
    dp.add_transition(stage=t, from_state=1, decision=2, to_state=1, probability=1/4, contribution=50 + 100*2)
    dp.add_transition(stage=t, from_state=1, decision=2, to_state=0, probability=1/2 + 1/4, contribution=50 + 100*2)
    
    # Produce 1
    dp.add_transition(stage=t, from_state=1, decision=1, to_state=1, probability=1/3, contribution=50 + 100*1)
    dp.add_transition(stage=t, from_state=1, decision=1, to_state=0, probability=2/3, contribution=50 + 100*1)
    
    # Produce 0
    dp.add_transition(stage=t, from_state=1, decision=0, to_state=1, probability=1, contribution=0)

# Transition probabilities and contributions from state n = 0
for t in range(number_of_stages - 1):
    # Produce 2
    dp.add_transition(stage=t, from_state=0, decision=2, to_state=0, probability=1, contribution=50 + 100*2)
    
    # Produce 1
    dp.add_transition(stage=t, from_state=0, decision=1, to_state=0, probability=1, contribution=50 + 100*1)
    
    # Produce 0
    dp.add_transition(stage=t, from_state=0, decision=0, to_state=0, probability=1, contribution=0)
        
# Boundary conditions
dp.add_boundary(state=0, value=0)
dp.add_boundary(state=1, value=1000)
dp.add_boundary(state=2, value=1000)

# Solve the stochastic dynamic program
value, policy = dp.solve()

# Examine value-to-go
print("Value-to-go function:")
print(value)

# Examine policy
print("\nCorresponding policy:")
print(policy)

Value-to-go function:
{(stage: 0, state: 0): 0
 (stage: 0, state: 1): 253.7037037037037
 (stage: 0, state: 2): 590.9722222222222
 (stage: 1, state: 0): 0
 (stage: 1, state: 1): 311.1111111111111
 (stage: 1, state: 2): 741.6666666666666
 (stage: 2, state: 0): 0
 (stage: 2, state: 1): 483.3333333333333
 (stage: 2, state: 2): 1000
 (stage: 3, state: 0): 0
 (stage: 3, state: 1): 1000
 (stage: 3, state: 2): 1000}

Corresponding policy:
{(stage: 0, state: 0): {0}
 (stage: 0, state: 1): {1}
 (stage: 0, state: 2): {2}
 (stage: 1, state: 0): {0}
 (stage: 1, state: 1): {1}
 (stage: 1, state: 2): {2}
 (stage: 2, state: 0): {0}
 (stage: 2, state: 1): {1}
 (stage: 2, state: 2): {0, 2}}


_Write your notes here. Double-click to edit._

_Solution._

* The minimum expected cost is $f_0(2) = 590,972.22$.

* The optimal policy is as follows:
    - Run 1: Produce 2. We will end up with 2, 1, or 0 missiles needed.
    - Run 2: If 2 missiles are still needed, produce 2. If 1 missile is still needed, produce 1. Otherwise produce 0. We will end up with 2, 1, or 0 missiles needed.
    - Run 3: If 2 missiles are needed, we're screwed; producing 2 or 0 missiles result in the same expected cost. If 1 missile is still needed, produce 1. Otherwise produce 0.

### Problem 2 (Farkas Investments, revisited)

You have recently been hired as a junior analyst at Farkas Investments. You have been given \$4 million to invest over the next 3 years. At the beginning of each of the next 3 years, you can invest in one of two investments: A or B.

| Investment | Cost (\$ millions) | Profit (\$ millions) | Probability |
|:-----------|:-------------------|:---------------------|:------------|
| A          | 3                  | 2                    | 0.5         |
|            |                    | -2                   | 0.5         |
| B          | 5                  | 3                    | 0.1         |
|            |                    | -1                   | 0.9         |

You are allowed to make at most one investment each year. Any additional money accumulated is left idle. You may not borrow money to invest; that is, you cannot buy into an investment if it costs more than you currently have.

Formulate a stochastic dynamic program to find an investment policy that maximizes the probability you will have at least \$10 million at the end of 3 years.

Once upon a time, for homework, you formulated this problem as a stochastic dynamic program.

1. Solve your dynamic program using `stochasticdp`.
2. Interpret the output of your stochastic dynamic program.

In [19]:
# Solution
# Number of stages
number_of_stages = 4

# List of states
states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# List of decisions
decisions = ['A', 'B', 'no investment']

# Initialize stochastic DP
dp = StochasticDP(number_of_stages, states, decisions, minimize=False)

# Transition probabilities and contributions
for t in range(number_of_stages - 1):
    for n in states:
        # Investment A
        if (n >= 3):
            dp.add_transition(stage=t, from_state=n, decision='A', to_state=min(n + 2, 10), probability=0.5, contribution=0)
            dp.add_transition(stage=t, from_state=n, decision='A', to_state=n - 2, probability=0.5, contribution=0)
        
        # Investment B
        if (n >= 5):
            dp.add_transition(stage=t, from_state=n, decision='B', to_state=min(n + 3, 10), probability=0.1, contribution=0)
            dp.add_transition(stage=t, from_state=n, decision='B', to_state=n - 1, probability=0.9, contribution=0)            
        
        # No investment
        dp.add_transition(stage=t, from_state=n, decision='no investment', to_state=n, probability=1, contribution=0)            
        
# Boundary conditions
for n in states:
    if n == 10:
        dp.add_boundary(state=n, value=1)
    else:
        dp.add_boundary(state=n, value=0)

# Solve stochastic DP
value, policy = dp.solve()

# Examine value-to-go
print("Value-to-go function:")
print(value)

# Examine policy
print("\nCorresponding policy:")
print(policy)

Value-to-go function:
{(stage: 0, state: 0): 0
 (stage: 0, state: 1): 0
 (stage: 0, state: 2): 0
 (stage: 0, state: 3): 0.025
 (stage: 0, state: 4): 0.125
 (stage: 0, state: 5): 0.125
 (stage: 0, state: 6): 0.25
 (stage: 0, state: 7): 0.325
 (stage: 0, state: 8): 0.625
 (stage: 0, state: 9): 0.625
 (stage: 0, state: 10): 1
 (stage: 1, state: 0): 0
 (stage: 1, state: 1): 0
 (stage: 1, state: 2): 0
 (stage: 1, state: 3): 0.0
 (stage: 1, state: 4): 0.0
 (stage: 1, state: 5): 0.05
 (stage: 1, state: 6): 0.25
 (stage: 1, state: 7): 0.25
 (stage: 1, state: 8): 0.5
 (stage: 1, state: 9): 0.55
 (stage: 1, state: 10): 1
 (stage: 2, state: 0): 0
 (stage: 2, state: 1): 0
 (stage: 2, state: 2): 0
 (stage: 2, state: 3): 0.0
 (stage: 2, state: 4): 0.0
 (stage: 2, state: 5): 0.0
 (stage: 2, state: 6): 0.0
 (stage: 2, state: 7): 0.1
 (stage: 2, state: 8): 0.5
 (stage: 2, state: 9): 0.5
 (stage: 2, state: 10): 1
 (stage: 3, state: 0): 0
 (stage: 3, state: 1): 0
 (stage: 3, state: 2): 0
 (stage: 3, stat

_Write your notes here. Double-click to edit._

_Solution._

* The maximum probability of reaching \\$10 million is $f_0(4) = 0.125$.

* The optimal policy is as follows:
    - Year 1: Invest in A. We will end up with either \\$2 or \\$6 million.
    - Year 2: If we end up with \\$2 million, don't invest (in fact, we can't, because we don't have enough money). If we end up with \\$6 million, invest in A. We can end up with \\$4 million or \$8 million.
    - Year 3: If we end up with \\$4 million, we can't reach \\$10 million by the end of the year. Since our objective in this problem is purely to maximize the probability we reach \\$10 million, either investing in A or not investing result in the same value-to-go (0). If we end up with \\$8 million, invest in A again.